# New-RscMssqlExport
In Rubrik, an export is a database recovery that you control all the parameters of. You can restore a database to a different instance or to the same instance as a different name. You can control the paths and filenames of the recovered database. You can control if the database should be fully recovered or left in NORECOVERY mode. You can control if you overwrite the target database or not. 

New-RscMssqlExport has multiple parameters, but the important ones to understand for most workflows are:
- RscMssqlDatabase: This will be the object returned from Get-RscMssqlDatabase
- RecoveryDateTime: This will be a value returned from Get-RscMssqlDatabaseRecoveryPoint
- TargetMssqlInstance: This will be the object returned from Get-RscMssqlInstance
- TargetDatabaseName: The name of the newly created database from the restore
- TargetDataPath & TargeLogPath: Used for the location of all data and log files of the database. This would be for the Simple Method in the UI.
- TargetFilePaths: Used for the location of all data and log files of the database. This would be the Advanced Method in the UI.  You are providing a hashtable with each file listed. You can get a list of files from Get-RscMssqlDatabaseFiles
- Overwrite: Used to denote if the target database should be overwritten 
- FinishRecovery: Ommitting this switch will restore the database into NORECOVERY mode. 

# Workflow
To start an Export (Recovery) of a database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the host id that the SQL Server is installed on. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscMssqlDatabaseRecoveryPoint** to get the fully formatted date that corresponds to the recovery point selected.
1. Use **Get-RscMssqlDatabaseFiles** to get the list of files from the backup. 
1. Use **Get-RscMssqlInstance** to get the instance id of the Target SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **New-RscMssqlExport** to start an in place restore of the database to the point in time provided. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [ ]:
Set-Location '/Users/Chris.Lumnah/Documents/rubrik-powershell-sdk'
Import-Module ./output/rubriksecuritycloud -force
# Import-Module RubrikSecurityCloud
Connect-RSC

## Get the Cluster ID

In [ ]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host Information

In [ ]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Instance Information

In [ ]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [ ]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance

## Get the fully formatted recovery point

In [ ]:
$RecoveryDateTime = Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -Latest

## Get the Target Instance Information

In [ ]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows
$TargetMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Export using the Simple Method

In [ ]:
$NewRscMssqlExport = @{
    RscMssqlDatabase = $RscMssqlDatabase
    RecoveryDateTime = $RecoveryDateTime
    TargetMssqlInstance = $TargetMssqlInstance
    TargetDatabaseName = "test"
    TargetDataPath = "c:\mnt\sqldata\"
    TargeLogPath = "c:\mnt\sqllogs\"
    Overwrite = $true
}
New-RscMssqlExport @NewRscMssqlExport

## Get list of files of the database from the backup

In [ ]:
$RscMssqlDatabaseFiles = Get-RscMssqlDatabaseFiles -RscMssqlDatabase $RscMssqlDatabase -RecoveryDateTime $RecoveryDateTime
$RscMssqlDatabaseFiles

## Create hash table of files

In [ ]:
$TargetFilePaths = @()
foreach($File in $RscMssqlDatabaseFiles){
    If ($File.FileType -eq 'MSSQL_DATABASE_FILE_TYPE_DATA'){
        $TargetDataPath = @{
            exportPath = $File.OriginalPath
            logicalName = $File.LogicalName
            newFilename = "Lumnah_Test.mdf"
        }
        $TargetFilePaths += $TargetDataPath
    }

    If ($File.FileType -eq 'MSSQL_DATABASE_FILE_TYPE_LOG'){
        $TargetLogPath = @{
            exportPath = $File.OriginalPath
            logicalName = $File.LogicalName
            newFilename = "Lumnah_Test_log.ldf"
        } 
        $TargetFilePaths += $TargetLogPath
    }
}
$TargetFilePaths

## Export using the Advanced Method

In [ ]:
$NewRscMssqlExport = @{
    RscMssqlDatabase = $RscMssqlDatabase
    RecoveryDateTime = $RecoveryDateTime
    TargetMssqlInstance = $TargetMssqlInstance
    TargetDatabaseName = "test"
    TargetFilePaths = $TargetFilePaths
    Overwrite = $true
}
New-RscMssqlExport @NewRscMssqlExport

## Track the status of the restore

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc